In [1]:
import os 
import zipfile as zf
from cryptography.fernet import Fernet as fn
import random
import string
import json
import time
import copy
import shutil

In [2]:
#global variables   ::Same as that of the above code
ver_control = 0
software_name = 0
creator = 0
section_UB = 0
error = 0
def versionControl(section = "Update Backup", ver="v0.0.2", soft_name="Project Ultra Backup", author="Anjal.P"):
    global ver_control, software_name, creator 
    ver_control = ver
    software_name = soft_name
    creator = author
    section_UB = section
    return ver_control, software_name, creator, section_UB

In [16]:
def errorSave(where_to_save, error_report, root_directory):
    global error
    where_to_save = where_to_save + "//"
    print("look at: ", where_to_save + "Error Report UB.txt")
    ver_control, software_name, creator, section = versionControl()
    date_error = str(time.localtime(time.time()).tm_mday) + "." + str(time.localtime(time.time()).tm_mon) + "." + str(time.localtime(time.time()).tm_year)
    if os.path.exists(where_to_save + "Error Report UB.txt")==False:
        try:
            with open(where_to_save + "Error Report UB.txt", 'a') as error_file:
                error_file.write("--"*50 + "\n")
                error_file.write("Errors :" + "[" +str(error) + "]" + "\n")
                error_file.write("On Running Restore: " + root_directory + "\n")
                error_file.write("Created On: " + date_error + "\n")
                error_file.write("Report: \n")
                for eachError in error_report:
                    error_file.write("     > " + eachError + " " + error_report[eachError])
                error_file.write("This is an automated report generated by " + str(software_name) + " " + str(ver_control) + " " + "Author: " + str(creator) + "\n")
                error_file.write("--"*50 + "\n")
                error_file.close()
        except Exception as e:
            print("Error even creating the error log .txt at: " + where_to_save + "Error Report UB.txt")
            print("Error Report: " + str(error_report))
            print("Error on errorSave: " + str(e))
    else:
        try:
            with open(where_to_save + "Error Report UB.txt", 'a') as error_file:
                error_file.write("--"*50 + "\n")
                error_file.write("Errors :" + "[" +str(error) + "]" + "\n")
                error_file.write("On Running Backup: " + root_directory + "\n")
                error_file.write("Created On: " + date_error + "\n")
                error_file.write("Report: \n")
                for eachError in error_report:
                    error_file.write("     > " + eachError + " " + error_report[eachError])
                error_file.write("This is an automated report generated by " + str(software_name) + " " + str(ver_control) + " " + "Author: " + str(creator) + "\n")
                error_file.write("--"*50 + "\n")
                error_file.close()
        except:
            print("Error even creating the error log .txt at: " + where_to_save + "Error Report UB.txt")
            print("Error Report: " + str(error_report))
            print("Error on errorSave: " + str(e))

In [5]:
def scanHistory(where_to_backup, ubmap_folder, working_dir): #see weather the where_to_backup has any .val file signif. prev. backup
    backup_history = {}   #stores the info on date of backup with keys: 0, 1, e.t.c
    backup_data = {}   #stores data on the backup folder and its corresponding .val file
    ubmap = []   #stores all the ubmap of each backup folder, from comparison from the .val and .ubmap files.
    history_count = 0
    data_count = 0
    backup_base = {}   #this is the place where the {folder:{"UB_h":{},".val":{}, ".ubmap":{}}}
    isUBPresent = 0
    global error
    error_report= {}
    for folder, subfolder, files in os.walk(where_to_backup):   #scanning the backed up folder and saving the data.
        if os.path.basename(folder)[:3]=="UB_":
            backup_history[history_count] = folder
            history_count += 1
        for each_file in files:
            if each_file[:3]=="UB_" and each_file[-4:]==".val":
                if os.path.basename(folder) not in backup_data.keys():
                    backup_data[os.path.basename(folder)] = []
                    backup_data[os.path.basename(folder)].append(folder.replace("\\", "//") + "//" + each_file)   #replace here to change \ to // as \ creates problem
                else:
                    backup_data[os.path.basename(folder)].append(folder + "//" + each_file)
    for ubfiles in os.listdir(ubmap_folder):          #taking care of the ub list session
        if ubfiles[-6:]==".UBmap" and ubfiles[:3]=="UB_":
            ubmap.append(ubfiles)
    for data in backup_data:
        backup_base[data] = {"UB_h":[], ".val":[], ".UBmap":[]}
        for history in backup_history:
            if data in os.listdir(backup_history[history]):
                if "oldBackups" not in backup_history[history]:    #avoid the oldBackups folder as this may mess with the results.
                    backup_base[data]["UB_h"].append(backup_history[history])
        for vsl_files in backup_data[data]:
            backup_base[data][".val"].append(vsl_files)
            for ub_find in ubmap:
                if os.path.basename(vsl_files)[:12]==ub_find[:12]:
                    backup_base[data][".UBmap"].append(ubmap_folder + ub_find)
    for mainFolder in backup_base:    #handling if a UBmap is missing.
        if len(backup_base[mainFolder][".UBmap"])!=len(backup_base[mainFolder][".val"]):
            error += 1
            error_report["ubmap missing: "] = "It seems that some UBmap files are missing from the ubmap directory."
            error_report["Missing files: "] = ".val files: " + str(backup_base[mainFolder][".val"]) + " .ubmap files: " + str(backup_base[mainFolder][".UBmap"])
            error_report["Plan of action: "] = "Proceed with the best option of backup all files for better recovery."
            errorSave(working_dir, error_report, ubmap_folder)
    return backup_base   #return the {folder:{"UB_h":[],".val":[], ".UBmap":[]}}
#this backup_base is the main dict-list where it get passed to the nect function which opens the root_directorty  nad 
#compars the files in the root_directory to ubmap if the match is found then the files are not written , moves to next
#and so on it continues and at times it amy find new data or modified data which may be required to backup
#in that case it remove old eub or sub files and copies the new files to the new history foler with data name
#not only this it also modidies the ubmap i.e. it deletes the enteries which has been remove and saves the 
#old ubmap as 

In [6]:
def directoryScan(root_directory):               #Scan the given directory ad reutrns the following variables 
    root_folders = {}                  #variable
    folder_no = 0
    root_files = {}
    file_no = 0
    root_size = 0 # in bytes.  
    files_properties = {}
    if os.path.isdir(root_directory)==False:     #define error in passes root_directory variable
        print("No such Directory exist.....")
    else:
        os.chdir(root_directory)
        for each_obj in os.listdir(root_directory):
            if os.path.isfile(each_obj)==True:    #if it is a file then do the below code
                root_files[file_no] = each_obj
                files_properties[file_no] = list(os.stat(each_obj))[6:]   #only contain 4 values size
                file_no = file_no + 1
            elif os.path.isdir(each_obj)==True:
                root_folders[folder_no] = each_obj                  #or if it is a folder :::
                folder_no += 1
    return root_folders, root_files, files_properties

#same function directoly from the createNewBackup code.

In [7]:
def prepToScan(root_directory):
    ver_control, software_name, creator, section = versionControl()
    folder_det = {}                                         #dict, where the map of the root folder goes.
    global error
    error_report = {}
    basefolder = os.path.basename(root_directory)             #select the base file
    split = root_directory.split("//")                       #seperate the selected directory w.r.t //
    index_basefolder = split.index(basefolder)                  #index the position of the basefile in the seperated list
    elim_sec = '//'.join(split[:index_basefolder]) + '//'              #join all the element of list except the basefile, which can be eliminated using 
    for folder, subfolder, files in os.walk(root_directory): #using index of the file. This gives the file to eliminate from the 'folder'.
        if ("$RECYCLE.BIN" or "found.000" or "System Volume Information") not in folder:
            os.chdir(root_directory)
            mod_folder = (folder.replace(elim_sec, '')).replace("\\", "//")
            folder_det[mod_folder] = {}
            try:
                folder_det[mod_folder]["fo_s"], folder_det[mod_folder]["fi_s"], folder_det[mod_folder]["fi_p"] = directoryScan(folder.replace("\\", "//") + "//")
            except Exception as e:
                error += 1
                print("Error in directoryScan...")
                error_report["directoryScan: "] = "Error: " + str(e)
                errorSave(working_dir, error_report, root_directory)
    return folder_det, basefolder

In [8]:
def modifyEngine(previous_backups, root_scan, root_basefolder):
    global error 
    error_report = {}
    ubstore = {}
    ubmap_root_present = {}   #only common files/folder from ubmap > root
    root_ubmap_present = {}    #only common files/folder from root > ubmap
    ubmap_root_delete = {}     #files present in ubmap but not in root so move the eub/sub to old_backup folder and delete such enteries from ubmap 
    root_ubmap_add = {}    #files present in root but not in ubmap so create the new backup folder and zip the new files with new ubmap of the created fioes.
    files_prop = {}
    len_count = 0
    len_zip_count = 0
    exceptionHandle = 0
    root_ubmap_add = copy.deepcopy(root_scan)
    root_ubmap_present = copy.deepcopy(root_scan)
    for mainFolder in previous_backups:
        if root_basefolder==mainFolder:
            ubstore = {}
            ubmap_root_present[mainFolder] = {}
            ubmap_root_delete[mainFolder] = {}
            #initialise and read the all ubmap assosiated with data_backup to ub_store
            for read_all_UBmap in previous_backups[mainFolder][".UBmap"]:
                if os.path.lexists(read_all_UBmap)==True:
                    try:
                        with open(read_all_UBmap, 'r') as ub_read:
                            try:
                                ubstore[read_all_UBmap] = json.load(ub_read)
                                ub_read.close()
                            except Exception as e:
                                ub_read.close()
                                error += 1
                                error_report["er_loadJSONUBmap_Engine"] = "Error in loading the UBmap: " + read_all_UBmap + " Error: " + str(e)
                                error_report["Plan of action: "] = "Procees with the update, while avoiding the UBmap" 
                                errorSave(working_dir, error_report, read_all_UBmap)
                    except Exception as e:
                        error += 1
                        error_report["er_readUBmap_Engine"] = "Error in reading UBmap " + read_all_UBmap + " Error: " + str(e)
                        error_report["Plan of action: "] = "Procees with the update, while avoiding the UBmap" 
                        errorSave(working_dir, error_report, read_all_UBmap)
            ubmap_root_present[mainFolder] = copy.deepcopy(ubstore)     # reading finished 
            ubmap_root_delete[mainFolder] = copy.deepcopy(ubstore)
            for eachUB in ubstore:
                for data_eachUB in ubstore[eachUB]:
                    if data_eachUB in root_scan.keys():
                        for each_data_folder in ubstore[eachUB][data_eachUB]["fo_s"]:
                            if ubstore[eachUB][data_eachUB]["fo_s"][each_data_folder] in root_scan[data_eachUB]["fo_s"].values():
                                try:
                                    ubmap_root_delete[mainFolder][eachUB][data_eachUB]["fo_s"].pop(each_data_folder)
                                except:
                                    exceptionHandle += 1
                            else:
                                try:
                                    ubmap_root_present[mainFolder][eachUB][data_eachUB]["fo_s"].pop(each_data_folder)
                                except:
                                    exceptionHandle += 1
                        for each_data_file in ubstore[eachUB][data_eachUB]["fi_s"]:
                            if ubstore[eachUB][data_eachUB]["fi_s"][each_data_file] in root_scan[data_eachUB]["fi_s"].values():
                                key_file = list(root_scan[data_eachUB]["fi_s"].keys())[list(root_scan[data_eachUB]["fi_s"].values()).index(ubstore[eachUB][data_eachUB]["fi_s"][each_data_file])]
                                if ubstore[eachUB][data_eachUB]["fi_p"][each_data_file][0]==root_scan[data_eachUB]["fi_p"][key_file][0] and ubstore[eachUB][data_eachUB]["fi_p"][each_data_file][2]==root_scan[data_eachUB]["fi_p"][key_file][2] and ubstore[eachUB][data_eachUB]["fi_p"][each_data_file][3]==root_scan[data_eachUB]["fi_p"][key_file][3]:
                                    try:
                                        ubmap_root_delete[mainFolder][eachUB][data_eachUB]["fi_s"].pop(each_data_file)
                                        ubmap_root_delete[mainFolder][eachUB][data_eachUB]["fi_p"].pop(each_data_file)
                                        ubmap_root_delete[mainFolder][eachUB][data_eachUB]["z_n"].pop(each_data_file)
                                        ubmap_root_delete[mainFolder][eachUB][data_eachUB]["k"].pop(each_data_file)
                                    except:
                                        exceptionHandle += 1
                                else:
                                    try:
                                        ubmap_root_present[mainFolder][eachUB][data_eachUB]["fi_s"].pop(each_data_file)
                                        ubmap_root_present[mainFolder][eachUB][data_eachUB]["fi_p"].pop(each_data_file)
                                        ubmap_root_present[mainFolder][eachUB][data_eachUB]["z_n"].pop(each_data_file)
                                        ubmap_root_present[mainFolder][eachUB][data_eachUB]["k"].pop(each_data_file)
                                    except:
                                        exceptionHandle += 1
                            else:
                                try:
                                    ubmap_root_present[mainFolder][eachUB][data_eachUB]["fi_s"].pop(each_data_file)
                                    ubmap_root_present[mainFolder][eachUB][data_eachUB]["fi_p"].pop(each_data_file)
                                    ubmap_root_present[mainFolder][eachUB][data_eachUB]["z_n"].pop(each_data_file)
                                    ubmap_root_present[mainFolder][eachUB][data_eachUB]["k"].pop(each_data_file)
                                except:
                                    exceptionHandle += 1
                    else:
                        try:
                            ubmap_root_present[mainFolder][eachUB].pop(data_eachUB)
                        except:
                            exceptionHandle += 1
                for data_Root in root_scan:
                    if data_Root in ubstore[eachUB]:
        #do not remove fo_s from the root_scan, as this is not logical.
                        for files_data_root in root_scan[data_Root]["fi_s"]:
                            if root_scan[data_Root]["fi_s"][files_data_root] in ubstore[eachUB][data_Root]["fi_s"].values():
                                rkey_file = list(ubstore[eachUB][data_Root]["fi_s"].keys())[list(ubstore[eachUB][data_Root]["fi_s"].values()).index(root_scan[data_Root]["fi_s"][files_data_root])]
                                if root_scan[data_Root]["fi_p"][files_data_root][0]==ubstore[eachUB][data_Root]["fi_p"][rkey_file][0] and root_scan[data_Root]["fi_p"][files_data_root][2]==ubstore[eachUB][data_Root]["fi_p"][rkey_file][2] and root_scan[data_Root]["fi_p"][files_data_root][3]==ubstore[eachUB][data_Root]["fi_p"][rkey_file][3]:
                                    try:
                                        root_ubmap_add[data_Root]["fi_s"].pop(files_data_root)
                                        root_ubmap_add[data_Root]["fi_p"].pop(files_data_root)
                                    except:
                                        exceptionHandle += 1
                                else:
                                    try:
                                        root_ubmap_present[data_Root]["fi_s"].pop(files_data_root)
                                        root_ubmap_present[data_Root]["fi_p"].pop(files_data_root)
                                    except:
                                        exceptionHandle += 1
                            else:
                                try:
                                    root_ubmap_present[data_Root]["fi_s"].pop(files_data_root)
                                    root_ubmap_present[data_Root]["fi_p"].pop(files_data_root)
                                except:
                                    exceptionHandle += 1
                    else:
                        try:
                            root_ubmap_present.pop(data_Root)
                        except:
                            exceptionHandle += 1
    return ubmap_root_present, ubmap_root_delete, root_ubmap_present, root_ubmap_add
                #for each_root in root_scan:    #checking root > ubmap realtion
                 #   if each_root in ubstore[eachUB].keys():       # check if root directory folders is present in ubmap.
# do not change any of the thing in this folder.
#during looping throw the dict always assign {**} and .copy() as used above                      
#ubmap_root_present is a dict, which return the files present in the both .ubmap and the root_scan i.e. files in root directory. this dict do not contain the files which is present in .ubmap but not in root directory.
#ubmap_root_delete : contains the files present in the .ubmap and not in root directory. useful in deleting the corresponding .eub or .sub files.
#root_ubmap_add : contains files present in root and not in the .ubmap : this is the list with reference to which new backup is created
#root_ubmap_present: which contain the files present in the root and the .ubmap : helps to verify the final output .eub files.

In [9]:
def zipEncryptFiles(fil_dict, dir_loc, nu_ub_files, where_to_B, working_dir):    #Convert the given folder file to a zip and then encrypt the same
    limit_zip = 300*1024*1024   #In bytes
    size_to = 200*1024*1024   #In bytes
    symbol_list = "~!`@#$%^&()_-+={[}];',"
    zip_name = {}
    key = {}                                  #the stuff where zip file get stored
    error = 0
    error_report = {}
    if len(fil_dict)!=0:                                         #creating the files to zip 
        for files in fil_dict:
            if len(fil_dict[files])!=0:
                os.chdir(dir_loc)                                     #changing the directory to the fodler in deal.
                error_report[files] = {}
                rand_range = random.randrange(8, 24)
                rand_name = ''.join(random.choices(string.ascii_letters + string.digits + symbol_list, k=rand_range))
                fi_name = rand_name + ".zip"
                try: 
                    rar_file = zf.ZipFile(working_dir + fi_name, 'w')
                    try:
                        rar_file.write(fil_dict[files], compress_type=zf.ZIP_DEFLATED)
                        rar_file.close()
                    except Exception as e:
                        rar_file.close()
                        error += 1
                        error_report["er_writing_zip"] = fil_dict[files] + " Error: " + str(e)
                        error_report["Plan of action"] = "Proceed with the next file..."
                        errorSave(working_dir, error_report, working_dir)
                except Exception as e:
                    error += 1
                    error_report["er_create_zip"] = working_dir + fi_name + " Error: " + str(e)
                    error_report["Plan of action"] = "Proceed with the next file..."
                    errorSave(working_dir, error_report, working_dir)
                key[files] = (fn.generate_key()).decode()   #as saving json do not read binary
                enc_key = fn(key[files].encode())
                os.chdir(working_dir)                           #encrypting the zip and writting to output directory(harddrive)
                if os.stat(fi_name).st_size>=limit_zip:                                    #if split needed
                    zip_name[files] = {}
                    num_split = os.stat(fi_name).st_size//size_to + 1 
                    try: 
                        with open (fi_name, 'rb') as split:
                            for volume in range(1, num_split + 1, 1):
                                rand_range = random.randrange(8, 24)
                                rand_name = ''.join(random.choices(string.ascii_letters + string.digits + symbol_list, k=rand_range))
                                split_name = where_to_B + "//" + rand_name + "."
                                zip_name[files][volume] = rand_name 
                                encr_spl_file = enc_key.encrypt(split.read(size_to))
                                try:
                                    with open (split_name + "sUB", 'wb') as writ_spl:
                                        try:
                                            writ_spl.write(encr_spl_file)
                                            writ_spl.close()
                                            nu_ub_files = nu_ub_files + 1  
                                        except Exception as e:
                                            writ_spl.close()
                                            error += 1
                                            error_report["er_write_sUB file"] = split_name + "sUB" + " Error: " + str(e)
                                            error_report["Plan of action: "] = "Proceed with next file to zip and encrypt"
                                            errorSave(working_dir, error_report, split_name)
                                except Exception as e:
                                    error += 1
                                    error_report["er_create_sUB file"] = split_name + "sUB" + " Error: " + str(e)
                                    error_report["Plan of action: "] = "Proceed with next file to zip and encrypt"
                                    errorSave(working_dir, error_report, split_name)
                        split.close()
                    except Exception as e:
                        error += 1
                        error_report["er_open_zip to encrypt"] = working_dir + "//" + fi_name + " Error: " + str(e)
                        error_report["Plan of action: "] = "Proceed with next file to zip and encrypt"
                        errorSave(working_dir, error_report, working_dir)
                    try:
                        os.remove(fi_name)                                                     #delete the zip file 
                    except Exception as e:
                        error += 1
                        error_report["er_deleting_zip"] = working_dir + fi_name + "Error: " + str(e)
                        errorSave(working_dir, error_report, working_dir)
                else:                                                                  #no split needed
                    zip_name[files] = fi_name[:-4]
                    try: 
                        with open (fi_name, 'rb') as no_split:
                            no_split_name = where_to_B + "//" + fi_name[:-4] + "."
                            encr_spl_file = enc_key.encrypt(no_split.read())
                            try:
                                with open (no_split_name + "eUB", 'wb') as writ_spl:
                                    try: 
                                        writ_spl.write(encr_spl_file)
                                        writ_spl.close()
                                        nu_ub_files = nu_ub_files + 1  
                                    except Exception as e:
                                        writ_spl.close()
                                        error += 1
                                        error_report["er_write_eUB file"] = no_split_name + "eUB" + " Error: " + str(e)
                                        error_report["Plan of action: "] = "Proceed with next file to zip and encrypt"
                                        errorSave(working_dir, error_report, no_split_name)
                            except Exception as e:
                                error += 1
                                error_report["er_create_eUB file"] = no_split_name + "eUB" + " Error: " + str(e)
                                error_report["Plan of action: "] = "Proceed with next file to zip and encrypt"
                                errorSave(working_dir, error_report, no_split_name) 
                        no_split.close()
                    except Exception as e:
                        error += 1
                        error_report["er_open_zip to encrypt"] = working_dir + "//" + fi_name + " Error: " + str(e)
                        error_report["Plan of action: "] = "Proceed with next file to zip and encrypt"
                        errorSave(working_dir, error_report, working_dir)
                    try:
                        os.remove(fi_name)
                    except Exception as e:
                        error += 1
                        error_report["er_deleting_zip"] = working_dir + fi_name + "Error: " + str(e)
                        errorSave(working_dir, error_report, working_dir)
    return zip_name, key, nu_ub_files

In [10]:
def filesBackup(dict_scan, root_directory, directory_to_backup, working_dir, ubmap_dir):
    ver_control, software_name, creator, section = versionControl()
    folder_det = dict_scan                                         #dict, where the map of the root folder goes.
    max_files = 100
    nu_ub_files = 0
    global error
    error_report = {}
    basefolder = os.path.basename(root_directory)                #select the base file
    split = root_directory.split("//")                       #seperate the selected directory w.r.t //
    index_basefolder = split.index(basefolder)                  #index the position of the basefile in the seperated list
    elim_sec = '//'.join(split[:index_basefolder]) + '//'              #join all the element of list except the basefile, which can be eliminated using 
    ubname = str(time.localtime(time.time()).tm_mday) + "." + str(time.localtime(time.time()).tm_mon) + "." + str(time.localtime(time.time()).tm_year)
    rand_folder = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
    rand_name = ''.join(random.choices(string.ascii_letters + string.digits, k=8))
    backup_to = directory_to_backup + "//UB_" + ubname + "//" + basefolder + "//" + rand_folder
    try:
        os.makedirs(backup_to)
    except Exception as e:
        error += 1
        error_report["er_create_folder: "] = backup_to
        error_report["Error: "] = str(e)
        print("Try running again...")
        return
    for mod_folder in folder_det: #using index of the file. This gives the file to eliminate from the 'folder'.
        try:
            folder_det[mod_folder]["z_n"], folder_det[mod_folder]["k"], nu_ub_files = zipEncryptFiles(folder_det[mod_folder]["fi_s"], elim_sec + mod_folder + "//", nu_ub_files, backup_to + "//", working_dir + "//")
        except Exception as e:
            error += 1
            error_report["er_zipEncryptFiles"] = "Error: " + str(e)
            errorSave(working_dir, error_report, root_directory)
        if nu_ub_files > max_files:
            nu_ub_files = 0
            rand_folder = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
            backup_to = directory_to_backup + "//UB_" + ubname + "//" + basefolder + "//" + rand_folder
            try:
                os.makedirs(backup_to)
            except Exception as e:
                error += 1
                error_report["er_create_folder: "] = backup_to
                error_report["Error: "] = str(e)
                print("Try running again...")
                return
    os.chdir(root_directory)
    try: 
        with open(ubmap_dir + "//" + "UB_" + rand_name + "_" + ubname + ".UBmap", 'w') as logfile:
            try:
                json.dump(folder_det, logfile)   #saving as a json file as json only help to read the dict from the file.
                logfile.close()
            except Exception as e:
                logfile.close()
                error += 1
                error_report["er_JSONdump_UBmap"] = "Error: " + str(e)
                error_report["Plan of action: "] = "Delete the created backup at: " + directory_to_backup + "//UB_" + ubname + basefolder + " and try again"
                errorSave(working_dir, error_report, ubmap_dir + "//")
                print("Delete the created backup at: " + directory_to_backup + "//UB_" + ubname + basefolder + " and try again")
                return
    except Exception as e:
        error += 1
        error_report["er_create_UBmap"] = "Error: " + str(e)
        error_report["Plan of action: "] = "Delete the created backup at: " + directory_to_backup + "//UB_" + ubname + basefolder + " and try again"
        errorSave(working_dir, error_report, ubmap_dir + "//")
        print("Delete the created backup at: " + directory_to_backup + "//UB_" + ubname + basefolder + " and try again")
        return
    try: 
        with open(directory_to_backup + "//UB_" + ubname + "//" + basefolder + "//" + "UB_" + rand_name + "_" + ubname + ".val", 'w') as verif_file:   #verification file... for unzip
            verif_file.write("V_File Code: " + str(rand_name) + "\n")
            verif_file.write("Created: " + str(ubname) + "\n")
            verif_file.write("Backup Of: " + root_directory + "\n")
            verif_file.write("Backed Up To: " + str(directory_to_backup + "//UB_" + ubname + "//" + basefolder) + "\n")
            verif_file.write("Mod_list: " + "[" + "\"" + "UB_" + rand_name + "_" + ubname + "\"" + "]")
            verif_file.write("\n" + "--"*25 + "\n")
            verif_file.write("This is an automated report generated by " + str(section) + " of " + str(software_name) + " " + str(ver_control) + " " + ", Author: " + str(creator))
            verif_file.close()
    except Exception as e:
        error += 1
        error_report["er_creating_.valfile"] = "Error: " + str(e)
        error_report["Plan of action: "] = "Manualy create a file named: " + "UB_" + rand_name + "_" + ubname + ".val" + " and leave it as empty and dave at" + directory_to_backup + "//UB_" + ubname + "//" + basefolder + "//"
        errorSave(working_dir, error_report, directory_to_backup + "//UB_" + ubname + "//" + basefolder + "//")
        print("Error: " + "Manualy create a file named: " + "UB_" + rand_name + "_" + ubname + ".val" + " and leave it as empty and dave at" + directory_to_backup + "//UB_" + ubname + "//" + basefolder + "//")
    return

In [11]:
def historyScan(root_folder): # takes the folder as root i.e. : ...UB_6.02.2020//data2
    history_scan = {}
    for folder, subfolder, files in os.walk(root_folder):
        for eachFiles in files:
            if os.path.basename(eachFiles)[-4:]==".eUB" or os.path.basename(eachFiles)[-4:]==".sUB":
                history_scan[eachFiles] = folder
    return history_scan

In [12]:
def moveDotUBFiles(move_dict, backup_directory, previous_map, randVal, working_dir):
    ver_control, software_name, creator, section = versionControl()
    global error
    error_report = {}
    files_move = {}
    scan_backup = {}
    prev_val = {}
    date = str(time.localtime(time.time()).tm_mday) + "." + str(time.localtime(time.time()).tm_mon) + "." + str(time.localtime(time.time()).tm_year)
    if os.path.isdir(backup_directory + "//" + "oldBackups")==False:
        try:
            os.makedirs(backup_directory + "//" + "oldBackups")
        except Exception as e:
            error += 1
            error_report["er_create_dir_oldBackups"] = backup_directory + "//" + "oldBackups"
            error_report["Error: "] = str(e)
            errorSave(working_dir, error_report, backup_directory)
            print("Unable to create new direcotry in: " + backup_directory)
            return
    for bFolder in move_dict:
        for ub_folder in previous_map[bFolder]["UB_h"]:       #maing the scan_backup for each history.... to one dict
            for ub_val in previous_map[bFolder][".val"]:     #making the .val file name to the prev_val so that the .val for the old can be generated
                prev_val[os.path.basename(ub_folder)] = ub_val
            scan_backup[os.path.basename(ub_folder)] = historyScan(backup_directory + "//" + os.path.basename(ub_folder) + "//" + bFolder)
            if os.path.isdir(backup_directory + "//" + "oldBackups" + "//" + os.path.basename(ub_folder) + "//" + bFolder)==False:
                try:
                    os.makedirs(backup_directory + "//" + "oldBackups" + "//" + os.path.basename(ub_folder) + "//" + bFolder)
                except Exception as e:
                    error += 1
                    error_report["er_create_oldBackup_mainFolder"] = backup_directory + "//" + "oldBackups" + "//" + os.path.basename(ub_folder) + "//" + bFolder
                    error_report["Error: "] = str(e)
                    errorSave(working_dir, error_report, backup_directory)
                    print("Error creating: " + str(backup_directory + "//" + "oldBackups" + "//" + os.path.basename(ub_folder) + "//" + bFolder))
                    return
        for eachUB in move_dict[bFolder]:             # taking all the files from the move dict.., which has to be moved to one dict
            ub_date = "UB_" + os.path.basename(eachUB)[:-6].replace(os.path.basename(eachUB)[:-6][:12], '')
            files_move[ub_date] = []
            for data_each in move_dict[bFolder][eachUB]:
                for files in move_dict[bFolder][eachUB][data_each]["fi_s"]:
                    if type(move_dict[bFolder][eachUB][data_each]["z_n"][files])==type({}):
                        for each_dict in move_dict[bFolder][eachUB][data_each]["z_n"][files].values():
                            files_move[ub_date].append(each_dict + ".sUB")
                    else:
                        files_move[ub_date].append(move_dict[bFolder][eachUB][data_each]["z_n"][files] + ".eUB")
        for each_history in files_move:      #moving the files form the move_dict folder, w.r.t the scan_backup.    
            if len(files_move[each_history])!=0:
                for files in files_move[each_history]:
                    if files in scan_backup[each_history].keys():
                        from_move = scan_backup[each_history][files] + "//" + files
                        to_move = backup_directory + "//" + "oldBackups" + "//" + each_history + "//" + bFolder + "//" + os.path.basename(scan_backup[each_history][files])
                        if os.path.isdir(to_move)==False:
                            try:
                                os.makedirs(to_move)
                            except Exception as e:
                                error += 1
                                error_report["er_create_dir"] = to_move
                                error_report["Error: "] = str(e)
                                errorSave(working_dir, error_report, backup_directory)
                                return
                        try:
                            shutil.move(from_move, to_move)
                        except Exception as e:
                            error += 1
                            error_report["er_moving_files"] = from_move + " to " + to_move
                            error_report["Error"] = str(e)
                            error_report["Plan of action: "] = "Try manually moving the files: from" + from_move + " to move to location: " + to_move
                            errorSave(working_dir, error_report, backup_directory)
                    try:
                        with open(backup_directory + "//" + "oldBackups" + "//" + each_history + "//" + bFolder + "//" + "old_" + randVal + "_" + os.path.basename(prev_val[each_history]), 'w') as oldVal:
                            oldVal.write("V_File Code: " + os.path.basename(prev_val[each_history])[3:11] + "\n")
                            oldVal.write("Part: " + each_history + " >> " + bFolder + " >> " + "eUB/sUB Files" + "\n")
                            oldVal.write("Created on: " + each_history[3:] + "\n")
                            oldVal.write("Modified on: " + date + "\n")
                            oldVal.write("Items Moved: " + "\n" + "     ")
                            oldVal.write(str(files_move[each_history]))
                            oldVal.write("\n" + "--"*25 + "\n")
                            oldVal.write("This is an automated report generated by " + str(section) + " of " + str(software_name) + " " + str(ver_control) + " " + ", Author: " + str(creator))
                            oldVal.write("\n\n\n")
                            oldVal.close()
                    except Exception as e:
                        error += 1
                        error_report["er_create_oldbackup_.val file"] = backup_directory + "//" + "oldBackups" + "//" + each_history + "//" + bFolder + "//" + "old_" + randVal + "_" + os.path.basename(prev_val[each_history])
                        error_report["Error: "] = str(e)
                        error_report["Plan of action: "] = "Try manually create the file: " + backup_directory + "//" + "oldBackups" + "//" + each_history + "//" + bFolder + "//" + "old_" + randVal + "_" + os.path.basename(prev_val[each_history]) + " without anything in it,"
                        errorSave(working_dir, error_report, backup_directory)
    return

In [13]:
def changeUBmap(ubmap_history, ubmap_dir, randUB, working_dir):
    global error
    error_report = {}
    if os.path.isdir(ubmap_dir + "//" + "oldUBmap")==False:
        try:
            os.makedirs(ubmap_dir + "//" + "oldUBmap")
        except Exception as e:
            error += 1
            error_report["er_create_oldUBmap folder"] = ubmap_dir + "//" + "oldUBmap"
            error_report["Error: "] = str(e)
            error_report["Plan of action : "] = "This error will make you to not rollback to previous backups to restore as be careful.."
            errorSave(working_dir, error_report, ubmap_dir)
            return
    for mainFolder in ubmap_history:
        for ub in ubmap_history[mainFolder]:
            basenameUB = os.path.basename(ub)
            rename_UB = "old_" + randUB + "_" + basenameUB
            try:
                shutil.move(ub, ubmap_dir + "//" + "oldUBmap" + "//" + rename_UB)
            except Exception as e:
                error += 1
                error_report["er_moving_ubmap file"] = ub
                error_report["Error: "] = str(e)
                error_report["Plan of action: "] = "The previous UBmap files are not moved to oldUBmap folder."
                errorSave(working_dir, error_report, ub)
            try:
                with open(ubmap_dir + "//" + basenameUB, 'w') as newUBmap:
                    try:
                        json.dump(ubmap_history[mainFolder][ub], newUBmap)
                        newUBmap.close()
                    except Exception as e:
                        newUBmap.close()
                        error += 1
                        error_report["er_dumpJSON_createNewUBmap"] = ubmap_dir + "//" + basenameUB
                        error_report["Error: "] = str(e)
                        error_report["Plan of action: "] = "Try create new backup."
                        errorSave(working_dir, error_report, ubmap_dir)
                        return
            except Exception as e:
                error += 1
                error_report["er_create_newUBmap"] = ubmap_dir + "//" + basenameUB
                error_report["Error: "] = str(e)
                error_report["Plan of action: "] = "Try create new backup."
                errorSave(working_dir, error_report, ubmap_dir)
                return
    return

In [14]:
def main(root_directory, directory_to_backup, working_dir, ubmap_dir):
    global error
    change_UBmap_history = {}
    move_eub_sub = {}
    verify = {}
    folder_det_new = {}
    perform_newBackup = False
    perform_modify = False
    error_report = {}
    try:
        dig_previous = scanHistory(directory_to_backup + "//", ubmap_dir + "//", working_dir)
    except Exception as e:
        error += 1
        error_report["scanHistory"] = "Error: " + str(e)
        errorSave(working_dir, error_report, root_directory)
        print("Error, scanHistory: Try the precess again")
        return
    try:
        root_scan, basefolder = prepToScan(root_directory)   #do not give the end "//" as this is not required
    except Exception as e:
        error += 1
        error_report["prepToScan"] = "Error: " + str(e)
        errorSave(working_dir, error_report, root_directory)
        print("Error, prepToScan: Try the precess again")
        return
    try:
        change_UBmap_history, move_eub_sub, verify, new_Backup = modifyEngine(dig_previous, root_scan, basefolder)
    except Exception as e:
        error += 1
        error_report["modifyEngine"] = "Error: " + str(e)
        errorSave(working_dir, error_report, root_directory)
        print("Error, modifyEngine: Try the precess again")
        return
    for data in new_Backup:    #rule out if there is new backup required required : i.e. if all the files are the same and there is not need of updatre
        for each_data in new_Backup[data]["fi_s"]:
            if len(new_Backup[data]["fi_s"][each_data])!=0:
                perform_newBackup = True
                break
    for mainFolder in move_eub_sub:   #looking for the file in the move_eub_sub, if this has no files oto replace then the perform_modify return a False
        for ub in move_eub_sub[mainFolder]:
            for data in move_eub_sub[mainFolder][ub]:
                for files in move_eub_sub[mainFolder][ub][data]["fi_s"]:
                    if len(move_eub_sub[mainFolder][ub][data]["fi_s"][files])!=0:
                        perform_modify = True
                        break   
    if perform_newBackup==True and perform_modify==True:  #perform new backup plus copy the modified files
        print("Found new files to backup and Updating existing backups.")
        try:
            filesBackup(new_Backup, root_directory, directory_to_backup, working_dir, ubmap_dir)
        except Exception as e:
            error += 1
            error_report["filesBackup"] = "Error: " + str(e)
            errorSave(working_dir, error_report, root_directory)
            print("Error in filesBackup: Try the precess again")
            return
        randUB = ''.join(random.choices(string.ascii_letters + string.digits, k=8))
        try:
            moveDotUBFiles(move_eub_sub, directory_to_backup + "//", dig_previous, randUB, working_dir)
        except Exception as e:
            error += 1
            error_report["moveDotUBFiles"] = "Error: " + str(e)
            errorSave(working_dir, error_report, root_directory)
            print("Error in filesBackup: Try the precess again")
            return
        try:
            changeUBmap(change_UBmap_history, ubmap_dir + "//", randUB, working_dir)
        except Exception as e:
            error += 1
            error_report["changeUBmap"] = "Error: " + str(e)
            errorSave(working_dir, error_report, root_directory)
            print("Error in changeUBmap: Try the precess again")
            return
    elif perform_modify==True:   #No new files found, so only modify the existing one
        print("Updating existing backups, No new Files to backup.")
        randUB = ''.join(random.choices(string.ascii_letters + string.digits, k=8))
        try:
            moveDotUBFiles(move_eub_sub, directory_to_backup + "//", dig_previous, randUB, working_dir)
        except Exception as e:
            error += 1
            error_report["moveDotUBFiles"] = "Error: " + str(e)
            errorSave(working_dir, error_report, root_directory)
            print("Error in filesBackup: Try the precess again")
            return
        try:
            changeUBmap(change_UBmap_history, ubmap_dir + "//", randUB, working_dir)
        except Exception as e:
            error += 1
            error_report["changeUBmap"] = "Error: " + str(e)
            errorSave(working_dir, error_report, root_directory)
            print("Error in changeUBmap: Try the precess again")
            return
    elif perform_newBackup==True:    #No modification of files found so only backup the mew ones
        print("Found new Files to backup, No update found.")
        try:
            filesBackup(new_Backup, root_directory, directory_to_backup, working_dir, ubmap_dir)
        except Exception as e:
            error += 1
            error_report["filesBackup"] = "Error: " + str(e)
            errorSave(working_dir, error_report, root_directory)
            print("Error in filesBackup: Try the precess again")
            return
    else:   # no new files or modification, so backup not necessery.
        print("All files already in backup.")
    return

In [15]:
def updateBackup(root_directory, directory_to_backup, working_dir, ubmap_dir):
    global error
    error_report = {}
    exception_folder = ["$RECYCLE.BIN", "found.000", "System Volume Information"]
    if root_directory.find("\\")!=-1:                    #change all the \ to //
        root_directory = root_directory.replace("\\", "//")
    if directory_to_backup.find("\\")!=-1:
        directory_to_backup = directory_to_backup.replace("\\", "//")
    if working_dir.find("\\")!=-1:
        working_dir = working_dir.replace("\\", "//")
    if ubmap_dir.find("\\")!=-1:
        ubmap_dir.replace("\\", "//")
    if os.path.isdir(root_directory)==False:    #check if all the folder exist.
        error += 1 
        print("root directory not found.....")
        error_report["Major error: "] = root_directory + " Not found"
        errorSave(working_dir, error_report, root_directory)
        return
    if os.path.isdir(directory_to_backup)==False:
        error += 1 
        print("directory to backup not found.....")
        error_report["Major error: "] = directory_to_backup + " Not found"
        errorSave(working_dir, error_report, root_directory)
        return
    if os.path.isdir(working_dir)==False:
        error += 1 
        print("working directory not found.....")
        error_report["Major error: "] = working_dir + " Not found"
        errorSave(working_dir, error_report, root_directory)
        return
    if os.path.isdir(ubmap_dir)==False:
        error += 1 
        print("ubmap directory not found.....")
        error_report["Major error: "] = ubmap_dir + " Not found"
        errorSave(working_dir, error_report, root_directory)
        return
    split = root_directory.split("//")   # make the code available for the root to be drive. i.e. each folder seperatly.
    if len(split)==2 and split[1]=='':   #condition where the root seems to be inside the dirve, so divide the task and 
        list_folders = os.listdir(root_directory)
        for each_root in list_folders:
            if each_root not in exception_folder:
                if os.path.isdir(root_directory + "//" + each_root)==True:
                    try:
                        main(root_directory + each_root, directory_to_backup, working_dir, ubmap_dir)
                    except Exception as e:
                        error += 1
                        print("Error: main function not working...")
                        error_report[each_root] = "error in main function: " + str(e)
                        errorSave(working_dir, error_report, root_directory + each_root)
                        return       
    else:
        try:
            main(root_directory, directory_to_backup, working_dir, ubmap_dir)
        except Exception as e:
            error += 1
            print("Error: main function not working.....")
            error_report[root_directory] = "error in main function: " + str(e)
            errorSave(working_dir, error_report, root_directory)
            return
    return "Success"